# Unsupervised Learning and Dimensionality Reduction

In [1]:
%matplotlib inline

The goal is for you to think about how these algorithms are  the same as, different from, and interact with your earlier work.

### Classification Data Review

In [2]:
from algo_evaluation.datasets import *

__Higgs Data Quick review__:

   - given outcomes of particle decays, detect Higgs boson; 
   
   - most of the supervised algorithms gave acceptable accuracy ranging from 0.8 - 0.9 far outperforming the backpropagation algorithm for neural networks (0.5)
   
   - learning weights with Randomized Optimization algorithms improved accuracy of the neural network to 0.7 and higher

In [3]:
higgs_data = load_higgs_train(sample_size=None, verbose=True, scale=True)

Size of the dataset: 68114
Number of features: 13
Number of positives (signal): 31894
Number of negatives (background): 36220


__Converters Data Quick review__:

   - given online user behavior and preferences predict whether or not ad display will result in convertion; 
   
   - in contrast to Higgs dataset, supervized algorithm all provided high accuracy >94%

In [4]:
bid_data = load_bidding_train(verbose=True)

Size of the dataset: 57970
Number of features: 15
Number of converters: 399
Number of non-converters: 54615
Number of leads: 2956


### K-means Clustering

In [5]:
from algo_evaluation.clustering import kmeans_eval
reload(kmeans_eval)

<module 'algo_evaluation.clustering.kmeans_eval' from '/Users/maestro/schoolspace/bag-of-algorithms/algo_evaluation/clustering/kmeans_eval.pyc'>

class sklearn.cluster.KMeans(n_clusters=8, init='k-means++', n_init=10, max_iter=300, tol=0.0001, precompute_distances=True, verbose=0, random_state=None, copy_x=True, n_jobs=1)

In [6]:
df_h = kmeans_eval.evaluate_k_means(higgs_data, estimators=kmeans_eval.higgs_estimators)

In [7]:
df_c = kmeans_eval.evaluate_k_means(bid_data, estimators=kmeans_eval.bid_estimators)

In [8]:
df_h

,estimator,v-measure,time
0,k_means_higgs_8,0.072969,6.541688
1,k_means_higgs_2,0.099647,0.789809


In [9]:
df_c

,estimator,v-measure,time
0,k_means_converters_8,0.000567,4.646239
1,k_means_converters_3,0.000627,0.941330
2,k_means_converters_2,0.000396,0.605682


http://scikit-learn.org/stable/modules/clustering.html#clustering-evaluation

You can choose your own measures of distance/similarity. Naturally, you'll have to justify your choices

### Expectation Maximization

In [12]:
from algo_evaluation.clustering import gmm_eval
reload(gmm_eval)

<module 'algo_evaluation.clustering.gmm_eval' from '/Users/maestro/schoolspace/bag-of-algorithms/algo_evaluation/clustering/gmm_eval.py'>

In [13]:
df_h = gmm_eval.evaluate_gmm(higgs_data, estimators=gmm_eval.higgs_estimators)

In [15]:
df_c = gmm_eval.evaluate_gmm(bid_data, estimators=gmm_eval.bid_estimators)

In [14]:
df_h

,estimator,v-measure,time
0,gmm_higgs_8,0.070865,14.141458
1,gmm_higgs_2,0.076695,3.851103


In [16]:
df_c

,estimator,v-measure,time
0,gmm_converters_2,0.000256,1.000598
1,gmm_converters_3,0.000355,1.190238
2,gmm_converters_8,0.002249,6.781609


### PCA

### ICA

### Randomized Projections

### Any other feature selection algorithm you desire

You are to run a number of experiments. Come up with at least two datasets

Run the clustering algorithms on the data sets and describe what you see

Apply the dimensionality reduction algorithms to the two datasets and describe what you see.

Reproduce your clustering experiments, but on the data after you've run dimensionality reduction on it.

Apply the dimensionality reduction algorithms to one of your datasets from assignment #1

and rerun your neural network learner on the newly projected data

Apply the clustering algorithms to the same dataset to which you just applied the dimensionality reduction algorithms (you've probably already done this), treating the clusters as if they were new features. In other words, treat the clustering algorithms as if they were dimensionality reduction algorithms. Again, rerun your neural network learner on the newly projected data.


A discussion of your datasets, and why they're interesting: If you're using the same datasets as before at least briefly remind us of what they are so we don't have to revisit your old assignment write-up.
explanations of your methods: How did you choose k?
a description of the kind of clusters that you got.
analyses of your results. Why did you get the clusters you did? Do they make "sense"? If you used data that already had labels (for example data from a classification problem from assignment #1) did the clusters line up with the labels? Do they otherwise line up naturally? Why or why not? Compare and contrast the different algorithms. What sort of changes might you make to each of those algorithms to improve performance? How much performance was due to the problems you chose? Be creative and think of as many questions you can, and as many answers as you can. Take care to justify your analysis with data explictly.
Can you describe how the data look in the new spaces you created with the various aglorithms? For PCA, what is the distribution of eigenvalues? For ICA, how kurtotic are the distributions? Do the projection axes for ICA seem to capture anything "meaningful"? Assuming you only generate k projections (i.e., you do dimensionality reduction), how well is the data reconstructed by the randomized projections? PCA? How much variation did you get when you re-ran your RP several times (I know I don't have to mention that you might want to run RP many times to see what happens, but I hope you forgive me)?
When you reproduced your clustering experiments on the datasets projected onto the new spaces created by ICA, PCA and RP, did you get the same clusters as before? Different clusters? Why? Why not?
When you re-ran your neural network algorithms were there any differences in performance? Speed? Anything at all?
It might be difficult to generate the same kinds of graphs for this part of the assignment as you did before; however, you should come up with some way to describe the kinds of clusters you get. If you can do that visually all the better.
